In [ ]:
#Import libraries
from numpy import *
from scipy.sparse import diags
from scipy.integrate import odeint
from matplotlib import animation as animation
from matplotlib import pyplot 
from pylab import *
from itertools import *

%matplotlib inline

In [ ]:
colors = ['#a55194','#7b4173','#d6616b','#ad494a','#e7ba52','#b5cf6b','#8ca252','#9c9ede','#5254a3']
# morados, rojos, amarillo, verdes y azules

anim_solutions = []

In [ ]:
#------------------------------------------------------------------------
#                        TIME AND SPACE ARRAYS
#------------------------------------------------------------------------
#end of time and space arrays
T=150
R=500

#size of the arrays
m=500                #time
n=1000                #space       

#time array
t=linspace(0,T,m,endpoint=True)

#space array
x=linspace(0,R,n,endpoint=True)

dt=t[1]-t[0]
dx=x[1]-x[0]


In [ ]:
dt

In [ ]:
dx

In [ ]:
#-------------------------------------------------------------------------
#                       VECTOR FIELD FOR THE SYSTEM
#-------------------------------------------------------------------------
# INPUTS
#  CN: array of (1,2*n) with the values for C and N
#  t: time array
#  n: size of the space array
#  D: difussion coefficient
# OUTPUT
#  CN: array with the value of the system at C and N

def field(CN,t,n,D):
    c=CN[0:n]
    n=CN[n:2*n]
    
    dc=c*(1-c-gama*n)-lamda*c*(1-K*n)
    dn=lamda*c*(1-K*n)
    
    if not D==0:
        dc+=D*dot(L,transpose(c))
    
    CN=[]
    
    CN.extend(dc)
    CN.extend(dn)
    
    return CN

In [ ]:
#Initial conditions for C and N
def initCN(x,t):   
    #Initial conditions
    C=array([exp(-0.4*k) for k in x])
    #C=array([1 if k<100 else 0 for k in x])
    N=array([0 for k in x ])
    
    return C,N

In [ ]:
#Constructs sparse matrix
def discretizationMatrix(dx,m,n):
    diagonals=[]
    
    diagonals.append(ones((1,n))[0]*(-2/pow(dx,2)))
    diagonals.append((ones((1,n-1))[0])*(1/pow(dx,2)))
    diagonals.append((ones((1,n-1))[0])*(1/pow(dx,2)))
    diagonals.append(zeros((1,n-2))[0])
    diagonals.append(zeros((1,n-2))[0])
    
    diagonals[2][n-2]=2/pow(dx,2)
    diagonals[2][n-2]=2/pow(dx,2)
    
    L=diags(diagonals,[0,-1,1,2,-2])*eye(n)
    
    return L

In [ ]:
def solveODE(n,t,D):
    
    sol=odeint(field,CN,t,args=(n,D))
    
    C_sol=sol[:,0:n]
    N_sol=sol[:,n:2*n]
    
    return [C_sol,N_sol]

In [ ]:
[C,N]=initCN(x,t)
L=discretizationMatrix(dx,m,n)
CN=[C[i] if i<n else N[i-n] for i in range(2*n)]

In [ ]:
lamda=0.1
K=3
gama=0

CN_sol=solveODE(n,t,1)
C_sol=CN_sol[0]
N_sol=CN_sol[1]


anim_solutions.append(CN_sol)

In [ ]:
fig, ax = plt.subplots(nrows=1,figsize=(10,5))

# remove right and top spine
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

ax.set_xlim(0,400)
ax.set_ylim(0,1.1)
ax.set_xlabel('x')
ax.set_ylabel('Densidad de células')

text(180,0.8,'Invasión completa',fontsize=12.0)
text(300,0.8,'Invasión nula',fontsize=12.0)

plot(x,C_sol[350],color=colors[8],linewidth=2)
plot(x,N_sol[350],color=colors[7],linestyle='--',linewidth=2)
savefig('./images/wavefront.eps',format='eps',dip=900)

# γ = 0

In [ ]:
gama=0

In [ ]:
#------------------------------------------------------------------------
#                    PLOT FOR DIFFERENT LAMBDAS
#------------------------------------------------------------------------
lamdas=[10,1,0.2,0.1]
clrs=[colors[0],colors[4],colors[5],colors[8]]
K=3

fig2, ax2 = plt.subplots(nrows=1,figsize=(10,4))

# remove right and top spine
ax2.spines['right'].set_visible(False)
ax2.spines['top'].set_visible(False)

ax2.set_xlim(0,350)
ax2.set_ylim(0,1.1)
ax2.set_xlabel("x")
ax2.set_ylabel("Densidad de células")


for (l,c) in zip(lamdas,clrs):
    lamda=l
    CN_sol=solveODE(n,t,1)
    
    dc_sol=CN_sol[0]
    dn_sol=CN_sol[1]
    
    plot(x,dc_sol[350],c,alpha=1,label='λ='+str(l),linewidth=2)
    plot(x,dn_sol[350],c,linestyle='--',alpha=0.8,linewidth=2)
    
legend(loc='best')
savefig('./images/lambdas_0.eps',format='eps',dpi=900)

In [ ]:
#------------------------------------------------------------------------
#                    PLOT FOR DIFFERENT K's
#------------------------------------------------------------------------
ks=[1,3,7,10]

lamda=0.3

fig3, ax3 = plt.subplots(nrows=1,figsize=(10,5))

# remove right and top spine
ax3.spines['right'].set_visible(False)
ax3.spines['top'].set_visible(False)

ax3.set_xlim(0,350)
ax3.set_ylim(0,1.1)
ax3.set_xlabel("x")
ax3.set_ylabel("Densidad de células")

for (k_i,c) in zip(ks,clrs):
    K=k_i
    
    CN_sol=solveODE(n,t,1)
    
    dc_sol=CN_sol[0]
    dn_sol=CN_sol[1]
    
    plot(x,dc_sol[350],c,alpha=1,label='K='+str(k_i))
    plot(x,dn_sol[350],c,linestyle='--',alpha=0.7)
    
legend(loc='best')
savefig('./images/ks_0.eps',format='eps',dpi=900)

### Wavespeed analysis

In [ ]:
lamda=0.1
K=3

CN_sol=solveODE(n,t,1)
    
dc=CN_sol[0]
dn = CN_sol[1]

#fixed point to approximate wavespeed
c_s=dc[1][200]

change_x=[x[200]]
index_x=[200]

for i in arange(0,500,10):
    c_sol = dc[i]

    x_pos = where(abs(c_s-c_sol)<0.05)
    
    if len(x_pos[0])>0:
        j  = x_pos[0][0]
        index_x.append(j)
        change_x.append(x[j])

#change in time 
dt_s = t[10]-t[1]
speed=[]

for i in range(1,len(change_x)):
    s = (change_x[i]-change_x[i-1])/dt_s
    speed.append(s)



In [ ]:
speed

## γ = 1

In [ ]:
gama=1

In [ ]:
#------------------------------------------------------------------------
#                    PLOT FOR DIFFERENT LAMBDAS
#------------------------------------------------------------------------
lamdas=[10,1,0.2,0.1]
K=3

fig4=figure(figsize=(10,5))
xlim(0,350)
ylim(0,1.5)
xlabel("x")
ylabel("Densidad de células")

for (l,c) in zip(lamdas,clrs):
    lamda=l
    CN_sol=solveODE(n,t,1)
    
    dc_sol=CN_sol[0]
    dn_sol=CN_sol[1]
    
    plot(x,dc_sol[200],color=c,alpha=0.8,label='λ='+str(l))
    plot(x,dn_sol[200],color=c,linestyle='--',alpha=0.7)
    
    if l==0.1:
        print('true')
        anim_solutions.append(CN_sol)
    
legend(loc='best')
#savefig('./images/lamdas_1.eps',format='eps',dpi=900)

In [ ]:
#------------------------------------------------------------------------
#                    PLOT FOR DIFFERENT K's
#------------------------------------------------------------------------
ks=[1.5,3,7,10]

lamda=0.3

fig5=figure(figsize=(10,5))
xlim(0,350)
ylim(0,1.2)
xlabel("x")
ylabel("Densidad de células")

for (k_i,c) in zip(ks,clrs):
    K=k_i
    
    CN_sol=solveODE(n,t,1)
    
    dc_sol=CN_sol[0]
    dn_sol=CN_sol[1]
    
    plot(x,dc_sol[200],c,alpha=1,label='K='+str(k_i))
    plot(x,dn_sol[200],c,linestyle='--',alpha=0.7)
    
    if k_i == 1.5:
        anim_solutions.append(CN_sol)
    
legend(loc='best')
#savefig('./images/ks_0.eps',format='eps',dpi=900)

### Wavespeed analysis

In [ ]:
lamda=0.2
K=3

CN_sol=solveODE(n,t,1)
    
dc=CN_sol[0]
dn = CN_sol[1]

#fixed point to approximate wavespeed
c_s=dc[1][200]

change_x=[x[200]]
index_x=[200]

for i in arange(0,500,10):
    c_sol = dc[i]

    x_pos = where(abs(c_s-c_sol)<0.05)
    
    if len(x_pos[0])>0:
        
        if x_pos[0][0]>2:
            j=x_pos[0][0]
        else:
            j=x_pos[0][1]
            
        index_x.append(j)
        change_x.append(x[j])

#change in time 
dt_s = t[10]-t[1]
speed=[]

for i in range(1,len(change_x)):
    s = (change_x[i]-change_x[i-1])/dt_s
    speed.append(s)



In [ ]:
speed

# Animations

In [ ]:
# Set up formatting for the movie files
Writer = animation.writers['ffmpeg']
data = dict(title='Wave animation', artist='andreamarin')
writer = Writer(fps=20, metadata=data)

In [ ]:
#normal wave animation for gamma = 0

sol=anim_solutions[0]
C_sol=sol[0]
N_sol=sol[1]


fig_anim=figure()
xlim(0, 500)
ylim(0, 1.2)
line_1, = plot(x,C_sol[0],color=colors[2], lw=2,label='Células precursoras')
line_2, = plot(x,N_sol[0],color=colors[2],alpha=0.6,lw=2,linestyle='--',label='Células diferenciadas')
legend(loc='best')
with writer.saving(fig_anim, "./videos/wave_0.mp4", dpi=600):
    for i in range(m):
        line_1.set_ydata(C_sol[i])
        line_2.set_ydata(N_sol[i])
        title('time = '+str(t[i]))
        writer.grab_frame()

In [ ]:
len(anim_solutions)

In [ ]:
#nonmonotonic wavefront for lambda = 0.1
sol=anim_solutions[2]
C_sol=sol[0]
N_sol=sol[1]

fig_anim2=figure()
xlim(0, 500)
ylim(0, 1.2)
line_1, = plot([],[],color=colors[8], lw=2,label='Células precursoras')
line_2, = plot([],[],color=colors[8],alpha=0.6,lw=2,linestyle='--',label='Células diferenciadas')
legend(loc='best')

with writer.saving(fig_anim2, "./videos/nmwave_1.mp4", dpi=600):
    for i in range(m):
        line_1.set_data(x,C_sol[i])
        line_2.set_data(x,N_sol[i])
        title('time = '+str(t[i]))
        writer.grab_frame()

In [ ]:
#nonmonotonic wavefront for K = 1.5
sol=anim_solutions[3]
C_sol=sol[0]
N_sol=sol[1]

fig_anim2=figure()
xlim(0, 500)
ylim(0, 1.2)
line_1, = plot([],[],color=colors[6], lw=2,label='Células precursoras')
line_2, = plot([],[],color=colors[6],alpha=0.6,lw=2,linestyle='--', label='Células diferenciadas')
legend(loc='best')
with writer.saving(fig_anim2, "./videos/nmwave_2.mp4", dpi=600):
    for i in range(m):
        line_1.set_data(x,C_sol[i])
        line_2.set_data(x,N_sol[i])
        title('time = '+str(t[i]))
        writer.grab_frame()